---

# Generic Conditional Laws for Random-Fields - via:

## Universal $\mathcal{P}_1(\mathbb{R})$-Deep Neural Model (Type A)

---

By: [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

# Training Algorithm:
---
## 1) Generate Data:
Generates the empirical measure $\sum_{n=1}^N \delta_{X_T(\omega_n)}$ of $X_T$ conditional on $X_0=x_0\in \mathbb{R}$ *($x_0$ and $T>0$ are user-provided)*.

## 2) Get "Sample Barycenters":
Let $\{\mu_n\}_{n=1}^N\subset\mathcal{P}_1(\mathbb{R}^d)$.  Then, the *sample barycenter* is defined by:
1. $\mathcal{M}^{(0)}\triangleq \left\{\hat{\mu}_n\right\}_{n=1}^N$,
2. For $1\leq n\leq \mbox{N sample barycenters}$: 
    - $
\mu^{\star}\in \underset{\tilde{\mu}\in \mathcal{M}^{(n)}}{\operatorname{argmin}}\, \sum_{n=1}^N \mathcal{W}_1\left(\mu^{\star},\mu_n\right),
$
    - $\mathcal{M}^{(n)}\triangleq \mathcal{M}^{(n-1)} - \{\mu^{\star}\},$
*i.e., the closest generated measure form the random sample to all other elements of the random sample.*

---
**Note:** *We simplify the computational burden of getting the correct classes by putting this right into this next loop.*

## 3) Train Deep Classifier:
$\hat{f}\in \operatorname{argmin}_{f \in \mathcal{NN}_{d:N}^{\star}} 
\sum_{x \in \mathbb{X}}
\, 
\mathbb{H}
\left(
    \operatorname{Softmax}_N\circ f(x)_n| I\left\{W_1(\hat{\mu}_n,\mu_x),\inf_{m\leq N} W_1(\hat{\mu}_m,\mu_x)\right\}
\right);
$
where $\mathbb{H}$ is the categorical cross-entropy.  

---
---
---
## Notes - Why the procedure is so computationally efficient?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

## Meta-Parameters

### Simulation

#### Ground Truth:
*The build-in Options:*
- rSDE 
- pfBM
- 2lnflow

In [1]:
# Option 1:
# groud_truth = "rSDE"
# Option 2:
groud_truth = "2lnflow"
## Option 3:
# groud_truth = "pfBM"

#### Grid Hyperparameter(s)

In [2]:
## Monte-Carlo
N_Euler_Maruyama_Steps = 10*2
N_Monte_Carlo_Samples = 10**2
N_Monte_Carlo_Samples_Test = 10**3 # How many MC-samples to draw from test-set?

# End times for Time-Grid
T_end = 1
T_end_test = 1.1


## Grid
N_Grid_Finess = 10*2
Max_Grid = 0.5
x_0 = 1

# Number of Centers (\hat{\mu}_s)
N_Quantizers_to_parameterize = 10**2

This option sets $\delta$ in $B_{\mathbb{R}\times [0,\infty)}(\hat{x}_n,\delta)$; where $\hat{x}_n\in \nu_{\cdot}^{-1}[\hat{\mu}]$.  N_measures_per_center sets the number of samples to draw in this ball...by construction the training set is $\delta$-bounded and $\nu_{(x,t)}$, for any such $x$ is $\omega_{\nu_{\cdot}}(\delta)$-bounded in $\mathcal{P}_1(\mathbb{R})$.

In [3]:
# Hyper-parameters of Cover
delta = 0.1
N_measures_per_center = 100

**Note**: Setting *N_Quantizers_to_parameterize* prevents any barycenters and sub-sampling.

#### Mode: Code-Testin Parameter(s)
- True: $\Rightarrow$ cross validation through grid of very mild parameters just to test hardward or software.
- False: $\Rightarrow$ run CV-grid.

In [4]:
trial_run = True

### Meta-parameters
Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.

In [5]:
test_size_ratio = .25

## Simulation from Rough SDE
Simulate via Euler-M method from:
$$ 
X_T^x = x + \int_0^T \alpha(s,X_s^x)ds + \int_0^T((1-\eta)\beta(s,X_s^s)+\eta\sigma_s^H)dW_s.
$$

### Drift

In [6]:
def alpha(t,x):
    return np.sin(math.pi*t)

### Volatility

In [7]:
def beta(t,x):
    return x

### Roughness Meta-parameters
 - Roughness is $H$,
 - Ratio_fBM_to_typical_vol is $\eta$.

In [8]:
Rougness = 0.9 # Hurst Parameter
Ratio_fBM_to_typical_vol = 0.5 # $\eta$ in equation above.

## Simulation from Measure-Valued $2$-Parameter Log-Gaussian Flow
$$
X_{t,x} \sim \log\text{-}\mathcal{N}\left(\alpha(t,x),\beta(t,x)\right).
$$

**Note:** *$\alpha$ and $\beta$ are specified below in the SDE Example*.

In [ ]:
# Run Backend
# %run Universal_Processes_Trainer_BACKEND.ipynb
exec(open('Universal_Processes_Trainer_BACKEND.py').read())

Using TensorFlow backend.
 38%|███▊      | 38/100 [00:00<00:00, 377.23it/s]

Deep Feature Builder - Ready
Deep Classifier - Ready
Building Training Set - 2-logNormal Ground-Truth


100%|██████████| 10/10 [00:00<00:00, 4818.82it/s]

Building Test Set - 2-logNormal Ground-Truth
Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:  2.1min finished


Epoch 1/200
316/316 [==============================] - 1s 2ms/step - loss: 3.0130 - accuracy: 0.1766
Epoch 2/200
316/316 [==============================] - 1s 2ms/step - loss: 1.4280 - accuracy: 0.4834
Epoch 3/200
316/316 [==============================] - 1s 3ms/step - loss: 1.0492 - accuracy: 0.6050
Epoch 4/200
316/316 [==============================] - 1s 3ms/step - loss: 0.8877 - accuracy: 0.6605
Epoch 5/200
316/316 [==============================] - 1s 4ms/step - loss: 0.8188 - accuracy: 0.6749
Epoch 6/200
316/316 [==============================] - 1s 3ms/step - loss: 0.7091 - accuracy: 0.7236
Epoch 7/200
316/316 [==============================] - 1s 4ms/step - loss: 0.6657 - accuracy: 0.7355
Epoch 8/200
316/316 [==============================] - 1s 3ms/step - loss: 0.6508 - accuracy: 0.7361
Epoch 9/200
316/316 [==============================] - 1s 2ms/step - loss: 0.6334 - accuracy: 0.7493
Epoch 10/200
316/316 [==============================] - 1s 2ms/step - loss: 0.6151 - accura

# Visualization
*From hereon out...do nothing and just let the backend sript run...the images and tables will load :).*

In [ ]:
# Run Backend
# %run Universal_Processes_Trainer_Visuals.ipynb
exec(open('Universal_Processes_Trainer_Visuals.py').read())

## Update User
We empirically estimate the standard error and confidence intervals or the relevant error distributions using the method of this paper: [Bootstrap Methods for Standard Errors, Confidence Intervals, and Other Measures of Statistical Accuracy - by: B. Efron and R. Tibshirani ](https://www.jstor.org/stable/2245500?casa_token=w_8ZaRuo1qwAAAAA%3Ax5kzbYXzxGSWj-EZaC10XyOVADJyKQGXOVA9huJejP9Tt7fgMNhmPhj-C3WdgbB9AEZdqjT5q_azPmBLH6pDq61jzVFxV4XxqBuerQRBLaaOFKcyr0s&seq=1#metadata_info_tab_contents)

### Training-Set Performance

In [ ]:
Type_A_Prediction

#### Predictions

In [ ]:
Type_A_Predictions_and_confidence.T

### Test-Set Performance

In [ ]:
Type_A_Prediction_test

#### Predictions

In [ ]:
Type_A_Predictions_and_confidence_test.T

### Model Complexity

In [ ]:
Model_Complexity

---

---
# Fin
---

---